# 📤 Entregar Trabajo

**Instrucciones:**
1. Completa todos los ejercicios en este notebook
2. Ejecuta todas las celdas
3. Ejecuta las 2 celdas de abajo para entregar

**NO requiere cuenta ni login** - solo ingresa tu nombre y listo.

In [ ]:
import json

# Check if we're in Pyodide (browser environment)
try:
    import pyodide
    from pyodide.http import pyfetch
    from pyodide.ffi import to_js
    print("✅ Pyodide detectado - Estamos en el navegador")
except ImportError:
    print("⚠️ Este notebook solo funciona en JupyterLite")
    print("   Visita: https://juakongdf.github.io/chaos-lab/")
    raise

SERVER_URL = "https://chaos-sub.vercel.app/submit"

async def submit_notebook(student_name):
    """Envía el notebook actual al servidor"""
    
    if not student_name or not student_name.strip():
        print("❌ Por favor ingresa tu nombre")
        return False
    
    print("⏳ Enviando tu trabajo...")
    
    # Get the current notebook content directly from JupyterLab using eval
    try:
        from js import eval as js_eval, JSON
        
        # First, let's see what's available
        debug_info = js_eval("""
        (function() {
            const info = {
                hasJupyterApp: typeof window.jupyterapp !== 'undefined',
                hasJupyterLab: typeof window.jupyterlab !== 'undefined',
                hasIPython: typeof window.IPython !== 'undefined',
                hasJupyter: typeof window.Jupyter !== 'undefined',
                windowKeys: Object.keys(window).filter(k => k.toLowerCase().includes('jupyter')).join(', ')
            };
            return JSON.stringify(info);
        })()
        """)
        print(f"🔍 Debug: {debug_info}")
        
        # Try multiple ways to get the notebook
        notebook_js = js_eval("""
        (function() {
            try {
                // Try method 1: window.jupyterapp
                let app = window.jupyterapp || window.jupyterlab;
                if (app && app.shell && app.shell.currentWidget) {
                    const widget = app.shell.currentWidget;
                    const context = widget.context;
                    const model = context.model;
                    return model.toJSON();
                }
                
                // Try method 2: IPython notebook
                if (window.IPython && window.IPython.notebook) {
                    return window.IPython.notebook.toJSON();
                }
                
                // Try method 3: Jupyter notebook
                if (window.Jupyter && window.Jupyter.notebook) {
                    return window.Jupyter.notebook.toJSON();
                }
                
                return null;
            } catch(e) {
                console.error('Error getting notebook:', e);
                return null;
            }
        })()
        """)
        
        if notebook_js is not None and str(notebook_js) != "null":
            notebook_json = JSON.stringify(notebook_js)
            notebook_content = json.loads(notebook_json)
            print("✅ Notebook capturado desde el editor")
        else:
            raise Exception("No se pudo acceder al notebook")
        
    except Exception as e:
        print(f"⚠️ No se pudo capturar desde el editor: {e}")
        print("📝 Enviando notebook vacío...")
        notebook_content = {
            "cells": [],
            "metadata": {},
            "nbformat": 4,
            "nbformat_minor": 5
        }
    
    # Prepare payload
    payload = {
        "student_name": student_name.strip(),
        "notebook": notebook_content
    }
    
    # Send to server using pyfetch (Pyodide's fetch wrapper)
    try:
        response = await pyfetch(
            SERVER_URL,
            method="POST",
            headers={"Content-Type": "application/json"},
            body=json.dumps(payload)
        )
        
        if response.ok:
            result = await response.json()
            print(f"✅ ¡Trabajo entregado exitosamente!")
            print(f"📄 Archivo: {result['filename']}")
            return True
        else:
            error_text = await response.text()
            try:
                error_data = json.loads(error_text)
                print(f"❌ Error del servidor: {error_data.get('error', 'Error desconocido')}")
            except:
                print(f"❌ Error HTTP {response.status}")
            return False
            
    except Exception as e:
        print(f"❌ Error de conexión: {e}")
        return False

print("✅ Sistema configurado")
print("👇 Para entregar, ejecuta la celda de abajo")

In [ ]:
# 📝 INGRESA TU NOMBRE Y EJECUTA ESTA CELDA

# Cambia "Tu Nombre Aquí" por tu nombre completo:
MI_NOMBRE = "Tu Nombre Aquí"

# Ejecuta esta celda para entregar tu trabajo
await submit_notebook(MI_NOMBRE)